In [ ]:
%pip install -r requirements.txt

Basic OpenAI Integration

In [ ]:
from openai import OpenAI
import json
import random
import string

#TODO: Remove keys
openai_key = "YOUR_OPENAI_KEY"
payi_api_key="YOUR_PAYI_API_KEY"

payi_oai_headers = {
    "xProxy-Forward-Authorization": "Bearer " + openai_key
}

#TODO: Replace with prod endpoint under the DNS
payi_oai_url = "https://apim-payi-qa-eastus.azure-api.net/api/v1/proxy/openai/v1"

oai_client = OpenAI(
    api_key=payi_api_key,
    base_url=payi_oai_url,
    default_headers=payi_oai_headers
)

response = oai_client.chat.completions.create(
    model="gpt-3.5-turbo", 
    messages=[{"role": "user", "content": "Say 'this is a test'"}],
)

completion = response.choices[0].message.content
print(completion)

proxy_result = response.proxy_result
print(json.dumps(proxy_result, indent=4))

In [ ]:
#TODO: Merge into the library directly
from typing import List, Dict

def create_budget_header_from_ids(budget_ids: List[str]) -> Dict[str, str]:
    if not isinstance(budget_ids, list):
        raise TypeError("budget_ids must be a list")
    
    valid_ids = [id.strip() for id in budget_ids if isinstance(id, str) and id.strip()]
    if valid_ids:
        return {"xProxy-Budget-IDs": ",".join(valid_ids)}
    return {}

def create_request_header_from_ids(request_ids: List[str]) -> Dict[str, str]:
    if not isinstance(request_ids, list):
        raise TypeError("request_tags must be a list")
    
    valid_ids = [id.strip() for id in request_ids if isinstance(id, str) and id.strip()]
    if valid_ids:
        return {"xProxy-Request-Tags": ",".join(valid_ids)}
    return {}

def create_headers(budget_ids: List[str] = None, request_tags: List[str] = None) -> Dict[str, str]:
    headers = {}
    if budget_ids is not None:
        headers.update(create_budget_header_from_ids(budget_ids))
    if request_tags is not None:
        headers.update(create_request_header_from_ids(request_tags))
    return headers

Use the Pay-i SDK to send a request with a request tag

In [ ]:
from payi import Payi
#from payi.lib.helpers import create_headers

#TODO: Replace with production endpoint
payi_client = Payi(
    payi_api_key=payi_api_key,
    base_url="https://apim-payi-qa-eastus.azure-api.net"
)

#TODO: Rename to the correct headers API
response = oai_client.chat.completions.create(
    model="gpt-3.5-turbo", 
    messages=[{"role": "user", "content": "Say 'this is a request tag test'"}],
    #extra_headers=payi_client.create_headers(request_ids=["x", "y"])
    extra_headers=create_headers(request_tags=["x", "y"])
)

completion = response.choices[0].message.content
print(completion)

proxy_result = response.proxy_result
print(json.dumps(proxy_result, indent=4))

Create a budget and make a request with that budget

In [ ]:
#Create a budget
budget_response = payi_client.budgets.create(
    #create a random name for the demo
    budget_name=''.join(random.choices(string.ascii_letters + string.digits, k=6)), 
    max=12.50, #$12.50 USD
    budget_type="Liberal",
    budget_response_type="Allow",
    budget_tags=["example_budget"]
)

budget_name = budget_response.budget.budget_name
budget_id = budget_response.budget.budget_id

print("Budget Created")
print(f"Budget Name: {budget_name}")
print(f"Budget ID: {budget_id}")

#Make a request using the new budget and request tags
response = oai_client.chat.completions.create(
    model="gpt-3.5-turbo", 
    messages=[{"role": "user", "content": "Say 'this is a budget test'"}],
    #extra_headers=payi_client.create_headers(request_ids=["x", "y"])
    extra_headers=create_headers(
        request_tags=["x", "y"], 
        budget_ids=[budget_id]
    )
)

completion = response.choices[0].message.content
print(completion)

proxy_result = response.proxy_result
print(json.dumps(proxy_result, indent=4))

See budget status

In [ ]:
response = payi_client.budgets.retrieve(budget_id=budget_id)
print(f"Budget Name: {response.budget.budget_name}")
print(f"Budget ID: {response.budget.budget_id}")
print(f"Budget Creation Timestamp: {response.budget.budget_creation_timestamp}")
print(f"Budget Tags: {response.budget.budget_tags}")
print(f"Budget Input Base Cost: {response.budget.totals.cost.input_cost.base}")
print(f"Budget Output Base Cost: {response.budget.totals.cost.output_cost.base}")
print(f"Budget Total Base Cost: {response.budget.totals.cost.total_cost.base}")

Make an ingest call with pre-computed token values

In [ ]:
response = payi_client.ingest.units(
    category="system.openai",
    resource="gpt-3.5-turbo",
    input=50,
    output=100,
    budget_ids= ",".join([budget_id]),
    request_tags = ",".join(["a", "b"])
)

print(f"Ingest request ID: {response.request_id}")
print(f"Input Base Cost: {response.cost.input.base}")
print(f"Output Base Cost: {response.cost.output.base}")
print(f"Total Base Cost: {response.cost.total.base}")

Reset a budget back to zero tracked cost

In [ ]:
response = payi_client.budgets.reset(budget_id=budget_id)
print(response.message)
print("State prior to reset: ")
print(f"Budget Name: {response.budget_history.budget_name}")
print(f"Budget ID: {response.budget_history.budget_id}")
print(f"Budget Tags: {response.budget_history.budget_tags}")
print(f"Budget Reset Timestamp: {response.budget_history.reset_date}")
print(f"Budget Input Base Cost: {response.budget_history.totals.cost.input.base}")
print(f"Budget Output Base Cost: {response.budget_history.totals.cost.output.base}")
print(f"Budget Total Base Cost: {response.budget_history.totals.cost.total.base}")

print("\nState after reset:")
response = payi_client.budgets.retrieve(budget_id=budget_id)
print(f"Budget Name: {response.budget.budget_name}")
print(f"Budget ID: {response.budget.budget_id}")
print(f"Budget Creation Timestamp: {response.budget.budget_creation_timestamp}")
print(f"Budget Tags: {response.budget.budget_tags}")
print(f"Budget Input Base Cost: {response.budget.totals.cost.input_cost.base}")
print(f"Budget Output Base Cost: {response.budget.totals.cost.output_cost.base}")
print(f"Budget Total Base Cost: {response.budget.totals.cost.total_cost.base}")

List and then delete all budgets

In [ ]:
response = payi_client.budgets.list()
for budget in response.items:
    print("Deleting budget with id:" + budget.budget_id)
    payi_client.budgets.delete(budget.budget_id)